In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import os

### 1. Data Processing

In [2]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(rescale=1.0/255, 
                              shear_range = 0.2,
                              zoom_range=0.2,
                              horizontal_flip = True)

# initialize the training generator
trainGen = trainAug.flow_from_directory('/Users/nimo/Desktop/Machine_Learning/7/hw/dataset_train',  # path
                                        target_size=(64, 64),  # all images will be resized to 64*64
                                        batch_size=32,
                                        class_mode='categorical')

Found 88 images belonging to 4 classes.


In [3]:
batchX, batchy = trainGen.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(32, 64, 64, 3), min=0.000, max=1.000


* What is the image shape of each training observation?
    * (64, 64, 3)
* How many total classes do we need to predict on? 
    * 4

### 2. Initial Classifier Build

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

In [5]:
classifier = Sequential()

# add the Conv2D layer
classifier.add(Convolution2D(filters = 32,
                             kernel_size = (3,3),
                             input_shape = (64, 64, 3),
                             activation = 'relu'))

# add the MaxPooling2D layer      
classifier.add(MaxPooling2D(pool_size = (2,2)))               
    
# add another Conv2D layer 

classifier.add(Convolution2D(filters = 64,
                             kernel_size = (3,3),
                             activation = 'relu'))

# add another MaxPooling2D layer
classifier.add(MaxPooling2D(pool_size = (2,2)))  

# add a Flatten layer 
classifier.add(Flatten())
               
# add a Dense layer    
classifier.add(Dense(units = 128, activation = 'relu'))     
               
# add a final Dense layer
classifier.add(Dense(units = 4, activation = 'softmax'))  
               
# compile   
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])           

In [6]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               1605760   
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 5

### 3. Model Runs

In [7]:
my_model = classifier.fit(trainGen, steps_per_epoch = 3, epochs = 3)

Epoch 1/3
3/3 [==============================] - 2s 454ms/step - loss: 1.4400 - accuracy: 0.3105
Epoch 2/3
3/3 [==============================] - 1s 363ms/step - loss: 0.9018 - accuracy: 0.6255
Epoch 3/3
3/3 [==============================] - 1s 351ms/step - loss: 0.4547 - accuracy: 0.9020


In [8]:
# save model
classifier.save('my_model.h5')
print("Saved model")

Saved model


In [9]:
# Predict
import os, glob
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')
print("Loaded model from disk")

# test data path
img_dir = "dataset_test" # Enter Directory of test set

# iterate over each test image
data_path = os.path.join(img_dir, '*g')
files = glob.glob(data_path)

# print the files in the dataset_test folder 
for f in files:
    print(f)
    
# make a prediction and add to results 
data = []
results = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result = model.predict(img)
    r = np.argmax(result, axis=1)
    results.append(r)

results

Loaded model from disk
dataset_test/C033.png
dataset_test/1022.png
dataset_test/4011.png
dataset_test/1053.png
dataset_test/6051.png
dataset_test/4053.png
dataset_test/C014.png
dataset_test/6023.png


[array([3]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([2]),
 array([3]),
 array([1])]

In [10]:
# training data path
train_img_dir = "dataset_train" 

# iterate over each training image
train_data_path = os.path.join(train_img_dir, '*g')
train_files = glob.glob(train_data_path)

# check category labels in training_set
trainGen.class_indices

{'category 1': 0, 'category 2': 1, 'category 3': 2, 'category 4': 3}

In [11]:
img = ['C033.png','1022.png','4011.png','1053.png','6051.png','4053.png','C014.png','6023.png']
predict = [3, 0, 1, 0, 2, 2, 3, 1]
actual = [3, 0, 2, 0, 1, 2, 3, 1]

result_33 = {'img': img, 'predict': predict, 'actual': actual}

In [12]:
df_33 = pd.DataFrame(data = result_33)
df_33

,img,predict,actual
0,C033.png,3,3
1,1022.png,0,0
2,4011.png,1,2
3,1053.png,0,0
4,6051.png,2,1
5,4053.png,2,2
6,C014.png,3,3
7,6023.png,1,1


In [13]:
# run other models 
my_model1 = classifier.fit(trainGen, steps_per_epoch = 1, epochs = 1)
print('--------------------------------------------------------------------------------------------------')
my_model2 = classifier.fit(trainGen, steps_per_epoch = 1, epochs = 2)
print('--------------------------------------------------------------------------------------------------')
my_model3 = classifier.fit(trainGen, steps_per_epoch = 1, epochs = 3)
print('--------------------------------------------------------------------------------------------------')
my_model4 = classifier.fit(trainGen, steps_per_epoch = 2, epochs = 4)
print('--------------------------------------------------------------------------------------------------')
my_model5 = classifier.fit(trainGen, steps_per_epoch = 2, epochs = 5)
print('--------------------------------------------------------------------------------------------------')
my_model6 = classifier.fit(trainGen, steps_per_epoch = 2, epochs = 6)
print('--------------------------------------------------------------------------------------------------')
my_model7 = classifier.fit(trainGen, steps_per_epoch = 3, epochs = 7)
print('--------------------------------------------------------------------------------------------------')
my_model8 = classifier.fit(trainGen, steps_per_epoch = 3, epochs = 8)
print('--------------------------------------------------------------------------------------------------')
my_model9 = classifier.fit(trainGen, steps_per_epoch = 5, epochs = 9)
print('--------------------------------------------------------------------------------------------------')
my_model10 = classifier.fit(trainGen, steps_per_epoch = 5, epochs = 10)

1/1 [==============================] - 0s 300ms/step - loss: 0.2734 - accuracy: 0.9583
--------------------------------------------------------------------------------------------------
Epoch 1/2
1/1 [==============================] - 0s 453ms/step - loss: 0.3076 - accuracy: 0.9688
Epoch 2/2
1/1 [==============================] - 0s 449ms/step - loss: 0.2038 - accuracy: 0.9375
--------------------------------------------------------------------------------------------------
Epoch 1/3
1/1 [==============================] - 0s 362ms/step - loss: 0.4336 - accuracy: 0.8750
Epoch 2/3
1/1 [==============================] - 0s 395ms/step - loss: 0.2582 - accuracy: 0.9062
Epoch 3/3
1/1 [==============================] - 0s 403ms/step - loss: 0.2269 - accuracy: 0.9688
--------------------------------------------------------------------------------------------------
Epoch 1/4
2/2 [==============================] - 1s 329ms/step - loss: 0.1405 - accuracy: 0.9688
Epoch 2/4
2/2 [===================

In [17]:
# create a manual process to generate repeated training data
n_batch = 5

n_epochs= 9

# loop through epochs
for e in range(n_epochs):
    print('Epoch', e)
    batches = 0
    # loop through batches
    for x_batch, y_batch in trainAug.flow_from_directory('dataset_train', target_size=(64, 64), 
                                                              batch_size = 32, class_mode = 'categorical', seed = 74): 
        classifier.fit(x_batch, y_batch)
        batches += 1
        if batches >= n_batch:
        # we need to break the loop by hand because the generator loops indefinitely
            break

Epoch 0
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 104ms/step - loss: 3.5886e-04 - accuracy: 1.0000
Epoch 1
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 80ms/step - loss: 3.4016e-04 - accuracy: 1.0000
Epoch 2
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 93ms/step - loss: 3.2437e-04 - accuracy: 1.0000
Epoch 3
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 87ms/step - loss: 3.0897e-04 - accuracy: 1.0000
Epoch 4
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 99ms/step - loss: 2.9368e-04 - accuracy: 1.0000
Epoch 5
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 143ms/step - loss: 2.7863e-04 - accuracy: 1.0000
Epoch 6
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 291ms/step - loss: 2.6431e-04 - accuracy: 1.0000
Epoch 7
Found 88 images belongi

In [16]:
n_batch = 5

n_epochs= 10

# loop through epochs
for e in range(n_epochs):
    print('Epoch', e)
    batches = 0
    # loop through batches
    for x_batch, y_batch in trainAug.flow_from_directory('dataset_train', target_size=(64, 64), 
                                                              batch_size = 32, class_mode = 'categorical', seed = 74): 
        classifier.fit(x_batch, y_batch)
        batches += 1
        if batches >= n_batch:
        # we need to break the loop by hand because the generator loops indefinitely
            break

Epoch 0
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 113ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 1
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 111ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 2
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 87ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 3
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 85ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 4
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 245ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 5
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 76ms/step - loss: 8.0022e-04 - accuracy: 1.0000
Epoch 6
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 107ms/step - loss: 5.7016e-04 - accuracy: 1.0000
Epoch 7
Found 88 images belonging to 4 classes.
1/

In [18]:
# create a final dataframe that combines the accuracy across each combination.
steps_per_epoch = [1,1,1,2,2,2,3,3,5,5]
epochs = [1,2,3,4,5,6,7,8,9,10]
accuracy = [0.9583,0.9688,0.9688,0.9643,0.9464,1.0000,0.9886,1.0000,1.0000,1.0000]
model_result = {'Steps per Epoch': steps_per_epoch, 
                'Epochs': epochs, 
                'Accuracy': accuracy}
df_model_result = pd.DataFrame(data = model_result)
df_model_result

,Steps per Epoch,Epochs,Accuracy
0,1,1,0.9583
1,1,2,0.9688
2,1,3,0.9688
3,2,4,0.9643
4,2,5,0.9464
5,2,6,1.0000
6,3,7,0.9886
7,3,8,1.0000
8,5,9,1.0000
9,5,10,1.0000


### Conceptual Questions: 

* Discuss the effect of the following on accuracy and loss (train & test): 
    * Increasing the steps_per_epoch -- increase the accuracy and decrease the loss. 
    * Increasing the number of epochs -- increase the accuracy and decrease the loss. 

* Name two uses of zero padding in CNN.
    * Control the shrinkage of dimension after applying filters larger than 1x1
    * Avoid loosing information at the boundaries

* What is the use of a 1 x 1 kernel in CNN? 
    * Decrease the number of feature maps.
    * Can be used for dimensionality reduction. (change the dimensionality in the filter space)
    * Alter the depth of the input volume. 
    * Reduce computation cost in a network, by reducing the depth.

* What are the advantages of a CNN over a fully connected DNN for this image classification problem?   
    * Fully connected networks tend to perform less and aren’t good for feature extraction. Plus they have a higher number of weights to train that results in high training time. On the other hand, CNNs are trained to identify and extract the best features from the images for the problem at hand with relatively fewer parameters to train. The latter layers of a CNN are fully connected because of their strength as a classifier.